In [1]:
!pip install transformers[sentencepiece] bitsandbytes sentencepiece accelerate datasets peft trl py7zr datasets

     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/43.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.8 kB ? eta -:--:--
     ---------------------------- ----------- 30.7/43.8 kB ? eta -:--:--
     -------------------------------------- 43.8/43.8 kB 142.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.0 kB ? eta -:--:--
     -------------------------------------- 42.0/42.0 kB 511.8 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- ----------------------------

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    unknown package:
        Expected sha256 52c1c7189a6ca006555a9663e544e75f40520a97a26e075411f9f9aca0771fcd
             Got        b75467a51b210801aa3aed43b5f7d28c6f5fb6a39acdae8cff9ee53c6672d6a5


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- ---------------------------------- 11.8/101.6 MB 343.9 kB/s eta 0:04:22
   ---- -------------------------------

In [2]:
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U datasets scipy ipywidgets matplotlib

In [1]:
from google.colab import output
output.enable_custom_widget_manager()

ModuleNotFoundError: No module named 'google'

In [ ]:
import torch
import transformers
import argparse
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from peft import (
        get_peft_model,
        prepare_model_for_kbit_training,
        LoraConfig
    )
from trl import SFTTrainer

In [ ]:
from huggingface_hub import login

login("HF_API_TOKEN ", add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                            load_in_8bit=True,
                                            device_map="auto"
                                            )
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# data = load_dataset("samsum")
# data_train, data_test, data_val = data["train"], data["test"], data["validation"]

# print(data_train, data_test, data_val)

# data_train[0]

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
}) Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
}) Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [ ]:
data_files = {"data": "/content/prompts_responses.7z"}
data = load_dataset("json", data_files=data_files)['data']

data = data.train_test_split(test_size=0.1, shuffle=True, seed=42)
data_train = data["train"]
data_val = data["test"]

In [ ]:
data_train[0]

{'prompt': 'I want to review the notes and records my academic advisor made regarding my progress and recommendations.',
 'response': "[Your Name]\n[Your Address]\n[City, Postal Code]\n[Email Address]\n[Phone Number]\n[Date]\n\n[Advisor's Name]\n[Advisor's Position]\n[Department Name]\n[University Name]\n\nDear Professor [Advisor's Last Name],\n\nI am writing to request a meeting to review the notes and records you have kept regarding my academic progress. I am particularly interested in gaining insights into your recommendations for my future academic and career endeavors.\n\nI would appreciate the opportunity to schedule a meeting at your earliest convenience. I am available on [list of available times]. Please let me know if any of these times work for you.\n\nThank you for your time and consideration.\n\nSincerely,\n[Your Signature]\n[Your Typed Name]"}

In [ ]:
def generate_prompt(prompt, response=None, eos_token="</s>"):
  instruction = "Write a letter for the following prompt:\n"
  input = f"{prompt}\n"
  response = f"Letter: {response + ' ' + eos_token if response else ''} "
  prompt = (" ").join([instruction, input, response])
  return prompt

print(generate_prompt(data_train[0]["prompt"], data_train[0]["response"]))

Write a letter for the following prompt:
 I want to review the notes and records my academic advisor made regarding my progress and recommendations.
 Letter: [Your Name]
[Your Address]
[City, Postal Code]
[Email Address]
[Phone Number]
[Date]

[Advisor's Name]
[Advisor's Position]
[Department Name]
[University Name]

Dear Professor [Advisor's Last Name],

I am writing to request a meeting to review the notes and records you have kept regarding my academic progress. I am particularly interested in gaining insights into your recommendations for my future academic and career endeavors.

I would appreciate the opportunity to schedule a meeting at your earliest convenience. I am available on [list of available times]. Please let me know if any of these times work for you.

Thank you for your time and consideration.

Sincerely,
[Your Signature]
[Your Typed Name] </s> 


In [ ]:
lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
 

In [ ]:
tokenizer.add_special_tokens({"pad_token": "<PAD>"})
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [ ]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [ ]:
output_dir = "cp"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
per_device_eval_batch_size = 4
eval_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 5e-4
max_grad_norm = 0.3
max_steps = 100 # play around with this
warmup_ratio = 0.03
evaluation_strategy="steps"
lr_scheduler_type = "constant"

training_args = transformers.TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            evaluation_strategy=evaluation_strategy,
            save_steps=save_steps,
            learning_rate=learning_rate,
            logging_steps=logging_steps,
            max_grad_norm=max_grad_norm,
            max_steps=max_steps,
            warmup_ratio=warmup_ratio,
            group_by_length=True,
            lr_scheduler_type=lr_scheduler_type,
            ddp_find_unused_parameters=False,
            eval_accumulation_steps=eval_accumulation_steps,
            per_device_eval_batch_size=per_device_eval_batch_size,
        )

In [ ]:
import json

# def formatting_func(example):
#     output = []
#     for i in range(len(example['prompt'])):
#       prompt = example["prompt"]
#       response = example["response"]
#       op = generate_prompt(prompt, response)
#       output.append(op)
#     return output

def formatting_func(prompt):
  output = []

  for d, s in zip(prompt["prompt"], prompt["response"]):
    op = generate_prompt(d, s)
    output.append(op)

  return output


trainer = SFTTrainer(
    model=model,
    train_dataset=data_train,
    eval_dataset=data_val,
    peft_config=lora_config,
    formatting_func=formatting_func,
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args
)

for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

trainer.train()
trainer.save_model(f"{output_dir}/final")

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bit

Step,Training Loss,Validation Loss
10,1.025400,0.755765
20,0.656400,0.634669
30,0.546000,0.605194
40,0.500100,0.600855
50,0.449300,0.630016


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain 

In [ ]:
from peft import PeftModel

peft_model_id = "cp/checkpoint-40"
peft_model = PeftModel.from_pretrained(model, peft_model_id, torch_dtype=torch.float16, offload_folder="lora_results/lora_7/temp")

In [ ]:
input_prompt = generate_prompt(data_train[50]["prompt"])
input_tokens = tokenizer(input_prompt, return_tensors="pt")["input_ids"].to("cuda")
with torch.cuda.amp.autocast():
    generation_output = peft_model.generate(
        input_ids=input_tokens,
        max_new_tokens=200, # play around with this
        do_sample=True,
        top_k=10,
        top_p=0.9,
        temperature=0.7,
        repetition_penalty=1.15,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
      )
op = tokenizer.decode(generation_output[0], skip_special_tokens=True)
print(op)

# [Your Name]
# [Your Address]
# [City, Postal Code]
# [Email Address]
# [Phone Number]
# [Date]

# Admissions Office
# [University Name]
# [University Address]

# Subject: Request for Program Change

# Dear Admissions Office,

# I am writing to formally request a change of program from Program X to Program Y. I was recently admitted to Program X, but after careful consideration, I have realized that my interests have shifted towards Program Y.

# Program Y aligns better with my current academic and career goals. I believe that the specialized courses and research opportunities offered in this program will provide me with the knowledge and skills necessary to pursue my aspirations.

# I have attached a copy of my updated transcript and resume for your review. I am confident that I possess the academic qualifications and drive to succeed in Program Y.

# I kindly request that you consider my request for a program change. I would be grateful for the opportunity to discuss further and provide any additional information that may be required.

# Thank you for your time and consideration.

# Sincerely,
# [Your Name]

Write a letter for the following prompt:
 I got admitted to program X but my interests have changed towards Y now. Can I change my program?
 Letter:  
Changing Your Program at University
Dear Sir or Madam,
I am writing this letter in order to inform you about my decision of changing from Computer Science (BSc) to Business Administration (BA). After careful consideration and consultation with my academic advisor, I decided that it would be beneficial for me to switch majors. This is because I have always been interested in business administration rather than computer science; therefore, pursuing an undergraduate degree in BA would allow me to further develop my skills as well as explore new opportunities within the field. In addition, there are many similarities between both programs which makes switching easier than one might think – such as course requirements being relatively similar despite different focus areas like programming languages versus financial statements analysis etc..
H

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!zip -r /content/cp.zip /content/cp
!mv /content/cp.zip /content/drive/MyDrive